<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Simulates a multi-crystal amplifier with BELLA Center's parameters</b>
<br>A 3-crystal configuration, with 4, 3, 3 passes per crystal
</div>

In [ ]:
# Laser pulse
num_laser_slices = 50

# Crystal
num_crystal_slices = 20

gain = 1 
radial_n2 = 0
prop_type = 'n0n2_srw'

beamsplitter_transmitted_frac = 0.4

In [ ]:
def extract_central_wavelength(laser_pulse):

    photon_e_ev = np.zeros(laser_pulse.nslice)
    photon_number = np.zeros(laser_pulse.nslice)

    for laser_index_i in np.arange(laser_pulse.nslice):
        photon_number[laser_index_i] = np.sum(laser_pulse.slice[laser_index_i].n_photons_2d.mesh)    
        photon_e_ev[laser_index_i] = laser_pulse.slice[laser_index_i].photon_e_ev

    weight = photon_number / np.sum(photon_number)
    wavelength = (const.h * const.c)/(const.e * photon_e_ev) * 1e9   

    return fit_gaussian(wavelength,weight)

In [ ]:
def fit_gaussian(wavelength, weight):
    
    wavelength = np.flip(wavelength)
    weight = np.flip(weight)
    
    cs = CubicSpline(wavelength, weight)
    wavelength_high_res = np.linspace(wavelength[0],wavelength[-1],len(wavelength)*5)
    weight_high_res = cs(wavelength_high_res)
    
    loc = np.argwhere(np.max(weight_high_res) == weight_high_res)
    
    plt.figure()
    plt.plot(wavelength,weight, 'k', label='Pulse Data')
    plt.plot(wavelength_high_res, weight_high_res, '--r', label='Cubic Spline Fit')
    plt.plot(wavelength_high_res[loc],weight_high_res[loc], '*', label='Central Wavelength')
    plt.legend()
    plt.show()

    return wavelength_high_res[loc]

In [ ]:
def plot_excited_states(crystal):
    long_excited_states, trans_excited_states = crystal.extract_excited_states()
    z = np.linspace(0,2.5,len(long_excited_states))
    x = np.linspace(-crystal.params.pop_inversion_mesh_extent,
                    crystal.params.pop_inversion_mesh_extent,
                    crystal.params.pop_inversion_n_cells)

    uti_plot.uti_plot2d1d(
        trans_excited_states.flatten(),
        [-crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_n_cells],
        [-crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_n_cells],
        0,
        0,
        ['Horizontal Position', 'Vertical Position', 'Total Num Excited States'],
        ['m', 'm', 'num'],
        True)
    
    fig = plt.figure()
    ax = fig.gca()
    plt.pcolormesh(x*(1e3), x*(1e3), trans_excited_states, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Total Num Excited States')
    plt.show()
    
    plt.figure()
    plt.plot(z,long_excited_states, 'k')
    plt.xlabel('Longitudinal Distance [cm]')
    plt.ylabel('Num Excited States')
    plt.show()

In [ ]:
def plot_all(laser_pulse):
    wfr_temp = laser_pulse.slice[0].wfr
    wfr_x = np.linspace(wfr_temp.mesh.xStart,wfr_temp.mesh.xFin,wfr_temp.mesh.nx)
    wfr_y = np.linspace(wfr_temp.mesh.yStart,wfr_temp.mesh.yFin,wfr_temp.mesh.ny)

    e_total_init = laser_pulse.extract_total_2d_elec_fields()
    intens_2d_init = 0.5 *const.c *const.epsilon_0 *(e_total_init.re**2.0 + e_total_init.im**2.0)
    intens0 = intens_2d_init.flatten()

    uti_plot.uti_plot2d1d(
        intens0,
        [wfr_temp.mesh.xStart, wfr_temp.mesh.xFin, wfr_temp.mesh.nx],
        [wfr_temp.mesh.yStart, wfr_temp.mesh.yFin, wfr_temp.mesh.ny],
        0,
        0,
        ['Horizontal Position', 'Vertical Position', 'Intensity'],
        ['m', 'm', 'ph/s/.1%bw/mm^2'],
        True)

    x_min = np.abs(-15.0 - wfr_x*1e3).argmin(axis=0) - 1
    x_max = np.abs(15.0 - wfr_x*1e3).argmin(axis=0) + 1
    
    with plt.style.context(('seaborn-poster')):
        fig = plt.figure(figsize=(4.6*1.5,3.6*1.5))
        ax = fig.gca()
        plt.pcolormesh(wfr_x[x_min:x_max]*(1e3), wfr_y[x_min:x_max]*(1e3), intens_2d_init[x_min:x_max,x_min:x_max], cmap=plt.cm.viridis, shading='auto')
        plt.colorbar()
        ax.set_ylabel(r'Vertical Position [mm]')
        ax.set_xlabel(r'Horizontal Position [mm]')
        # plt.xlim(-15,15)
        # plt.ylim(-15,15)
        ax.set_title('Intensity')
        plt.show()
    
    phase_2d_init = laser_pulse.extract_total_2d_phase()
    phase0 = phase_2d_init.flatten()

    uti_plot.uti_plot2d1d(
        phase0,
        [wfr_temp.mesh.xStart, wfr_temp.mesh.xFin, wfr_temp.mesh.nx],
        [wfr_temp.mesh.yStart, wfr_temp.mesh.yFin, wfr_temp.mesh.ny],
        0,
        0,
        ['Horizontal Position', 'Vertical Position', 'Phase'],
        ['m', 'm', ''],
        True)

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure(figsize=(4.6*1.5,3.6*1.5))
        ax = fig.gca()
        plt.pcolormesh(wfr_x[x_min:x_max]*(1e3), wfr_y[x_min:x_max]*(1e3), phase_2d_init[x_min:x_max,x_min:x_max], cmap=plt.cm.viridis, shading='auto')
        plt.colorbar()
        ax.set_ylabel(r'Vertical Position [mm]')
        ax.set_xlabel(r'Horizontal Position [mm]')
        # plt.xlim(-15,15)
        # plt.ylim(-15,15)
        ax.set_title('Phase')
        plt.show()

#### Imports

In [ ]:
import sys, copy
import numpy as np
from pykern.pkcollections import PKDict
import time

from scipy import optimize
from scipy.interpolate import splrep, splev
from scipy.interpolate import CubicSpline
from scipy import special

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rsmath import lct as rslct
from rslaser.pulse import pulse
from rslaser.optics import crystal
from rslaser.optics import drift
from rslaser.optics import lens
import rslaser.utils.srwl_uti_data as srwutil
import uti_plot

import srwlib

import scipy.constants as const
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline
from scipy import special

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['pcolor.shading'] ='auto'
%matplotlib inline

### Laser Pulse

In [ ]:
w0 = 3.056e-3 
l_scale = np.sqrt(np.pi)*w0

params = PKDict(
    num_sig_long = 6,
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = num_laser_slices,
    pulseE      = 0.002,
    tau_fwhm    = 300.0e-12 * np.sqrt(2.), # was /sqrt(2)
    tau_0       = 35.0e-15 * np.sqrt(2.),
    sigx_waist  = w0/np.sqrt(2.0), # was /2
    sigy_waist  = w0/np.sqrt(2.0),
    num_sig_trans=10,
    nx_slice=512,
    phase_flatten_cutoff=0.7,
)

### Crystal 1 ('Amp2')

In [ ]:
crystal_params_1 = PKDict(
    length      = 0.025, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = [0.00 for _ in range(num_crystal_slices)],
    l_scale     = np.sqrt(np.pi)*w0,
    pop_inversion_n_cells=128,
    pop_inversion_mesh_extent=0.01/2.0,  # [m]
    pop_inversion_crystal_alpha=106.4,  # [1/m], 1.064 1/cm
    pop_inversion_pump_waist=3.82e-3,  # [m]
    pop_inversion_pump_wavelength=532.0e-9,  # [m]
    pop_inversion_pump_energy=0.82,  # [J], pump laser energy onto the crystal
    pop_inversion_pump_type="dual",
    pop_inversion_pump_gaussian_order=8,
    pop_inversion_pump_rep_rate=1.0,
)

crystal_1 = crystal.Crystal(crystal_params_1)
n0_slice_array, n2_slice_array1, full_crystal_abcd_mat_1 = crystal_1.calc_n0n2(set_n=True, method="fenics", heat_load="hog")

if prop_type == 'abcd_lct':
    crystal_params_1.A = full_crystal_abcd_mat_1[0][0]
    crystal_params_1.B = full_crystal_abcd_mat_1[0][1]
    crystal_params_1.C = full_crystal_abcd_mat_1[1][0]
    crystal_params_1.D = full_crystal_abcd_mat_1[1][1]
    crystal_params_1.nslice = 1
    crystal_params_1.n0 = [1.76]
    crystal_params_1.n2 = [0.00]
    crystal_1 = crystal.Crystal(crystal_params_1)
    
plot_excited_states(crystal_1)

### Crystal 2 ('Amp3')

In [ ]:
crystal_params_2 = PKDict(
    length      = 0.025, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = [0.00 for _ in range(num_crystal_slices)],
    l_scale     = np.sqrt(np.pi)*w0,
    pop_inversion_n_cells=128,
    pop_inversion_mesh_extent=0.015/2.0,  # [m]
    pop_inversion_crystal_alpha=106.4,  # [1/m], 1.064 1/cm
    pop_inversion_pump_waist=7.3e-3,  # [m]
    pop_inversion_pump_wavelength=532.0e-9,  # [m]
    pop_inversion_pump_energy=2.75,  # [J], pump laser energy onto the crystal
    pop_inversion_pump_type="dual",
    pop_inversion_pump_gaussian_order=8,
    pop_inversion_pump_rep_rate=1.0,
)

crystal_2 = crystal.Crystal(crystal_params_2)
n0_slice_array, n2_slice_array2, full_crystal_abcd_mat_2 = crystal_2.calc_n0n2(set_n=True, method="fenics", heat_load="hog")

if prop_type == 'abcd_lct':
    crystal_params_2.A = full_crystal_abcd_mat_2[0][0]
    crystal_params_2.B = full_crystal_abcd_mat_2[0][1]
    crystal_params_2.C = full_crystal_abcd_mat_2[1][0]
    crystal_params_2.D = full_crystal_abcd_mat_2[1][1]
    crystal_params_2.nslice = 1
    crystal_params_2.n0 = [1.76]
    crystal_params_2.n2 = [0.00]
    crystal_2 = crystal.Crystal(crystal_params_2)
    
plot_excited_states(crystal_2)

### Crystal 3 ('Amp4')

In [ ]:
crystal_params_3 = PKDict(
    length      = 0.025, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = [0.00 for _ in range(num_crystal_slices)],
    l_scale     = np.sqrt(np.pi)*w0,
    pop_inversion_n_cells=128,
    pop_inversion_mesh_extent=0.03/2.0,  # [m]
    pop_inversion_crystal_alpha=106.4,  # [1/m], 1.064 1/cm
    pop_inversion_pump_waist=15.4e-3,  # [m]
    pop_inversion_pump_wavelength=532.0e-9,  # [m]
    pop_inversion_pump_energy=7.0,  # [J], pump laser energy onto the crystal
    pop_inversion_pump_type="dual",
    pop_inversion_pump_gaussian_order=8,
    pop_inversion_pump_rep_rate=1.0,
)

crystal_3 = crystal.Crystal(crystal_params_3)
n0_slice_array, n2_slice_array3, full_crystal_abcd_mat_3 = crystal_3.calc_n0n2(set_n=True, method="fenics", heat_load="hog") #method="analytical"

if prop_type == 'abcd_lct':
    crystal_params_3.A = full_crystal_abcd_mat_3[0][0]
    crystal_params_3.B = full_crystal_abcd_mat_3[0][1]
    crystal_params_3.C = full_crystal_abcd_mat_3[1][0]
    crystal_params_3.D = full_crystal_abcd_mat_3[1][1]
    crystal_params_3.nslice = 1
    crystal_params_3.n0 = [1.76]
    crystal_params_3.n2 = [0.00]
    crystal_3 = crystal.Crystal(crystal_params_3)
    
plot_excited_states(crystal_3)

## Drift Matrix

In [ ]:
# Drift length [m]
L_Drift = 2.75/2.0 # [m]

# instantiate an srw drift
e_drift_srw = lens.Drift_srw(L_Drift)

# instantiate an lct drift
e_drift_lct = lens.Drift_lct(L_Drift, l_scale)

***
## Initial Intensity and Phase

In [ ]:
thisPulse_initial = pulse.LaserPulse(params)
thisPulse_initial.zero_phase()
plot_all(thisPulse_initial)

In [ ]:
print(thisPulse_initial.calc_total_energy())

In [ ]:
rayleigh_length = np.pi * (thisPulse_initial.sigx_waist)**2.0 / (thisPulse_initial._lambda0)
print('Rayleigh Length:', round(rayleigh_length,3), ' m')
print('RMS bunch length:', round(thisPulse_initial.sig_s,3), ' m')

In [ ]:
pass_number = np.arange(11)
central_wavelength = np.zeros(len(pass_number))
central_wavelength[0] = extract_central_wavelength(thisPulse_initial)

***
## 4 Passes Through First Crystal ('Amp2')

In [ ]:
thisPulse_firstCrystal = copy.deepcopy(thisPulse_initial)
num_passes = 4

for prop_index in np.arange(num_passes):
    print('Pass ',prop_index+1)
    
    thisPulse_firstCrystal = e_drift_srw.propagate(thisPulse_firstCrystal)
    
    # Crystal
    thisPulse_firstCrystal = crystal_1.propagate(thisPulse_firstCrystal, prop_type, gain, radial_n2)
    plot_excited_states(crystal_1)
    
    thisPulse_firstCrystal = e_drift_srw.propagate(thisPulse_firstCrystal)
    
    if prop_index < (num_passes -1):
        thisPulse_firstCrystal.ideal_mirror_180()

    central_wavelength[1 + prop_index] = extract_central_wavelength(thisPulse_firstCrystal)
        
plot_all(thisPulse_firstCrystal)
print(thisPulse_firstCrystal.calc_total_energy())

***
## Beamsplitter

In [ ]:
beamsplitter = lens.Beamsplitter(beamsplitter_transmitted_frac)

thisPulse_beamsplitter = copy.deepcopy(thisPulse_firstCrystal)
thisPulse_beamsplitter = beamsplitter.propagate(thisPulse_beamsplitter)
plot_all(thisPulse_beamsplitter)
print(thisPulse_beamsplitter.calc_total_energy())

***
## Telescope 1

In [ ]:
f1 = -0.1
f2 = 0.2
d1 = 1.9
d2 = 0.1
d3 = 0.54

telescope_1 = lens.Telescope_lct(f1, f2, d1, d2, d3, l_scale)

thisPulse_telescope1 = copy.deepcopy(thisPulse_beamsplitter)
thisPulse_telescope1 = telescope_1.propagate(thisPulse_telescope1)
plot_all(thisPulse_telescope1)
print(thisPulse_telescope1.calc_total_energy())

***
## 3 Passes Through Second Crystal ('Amp3')

In [ ]:
thisPulse_secondCrystal = copy.deepcopy(thisPulse_telescope1)
num_passes = 3

for prop_index in np.arange(num_passes):
    print('Pass ', prop_index+1)

    thisPulse_secondCrystal = e_drift_srw.propagate(thisPulse_secondCrystal)
    
    # Crystal
    thisPulse_secondCrystal = crystal_2.propagate(thisPulse_secondCrystal, prop_type, gain, radial_n2)
    plot_excited_states(crystal_2)
    
    thisPulse_secondCrystal = e_drift_srw.propagate(thisPulse_secondCrystal)
    
    if prop_index < (num_passes -1):
        thisPulse_secondCrystal.ideal_mirror_180()

    central_wavelength[5 + prop_index] = extract_central_wavelength(thisPulse_secondCrystal)
        
plot_all(thisPulse_secondCrystal)
print(thisPulse_secondCrystal.calc_total_energy())

***
## Telescope 2

In [ ]:
f1 = -0.3
f2 = 0.6
d1 = 1.0
d2 = 0.3
d3 = 0.66

telescope_2 = lens.Telescope_lct(f1, f2, d1, d2, d3, l_scale)

thisPulse_telescope2 = copy.deepcopy(thisPulse_secondCrystal)
thisPulse_telescope2 = telescope_2.propagate(thisPulse_telescope2)
plot_all(thisPulse_telescope2)
print(thisPulse_telescope2.calc_total_energy())

***
## 3 Passes Through Third Crystal ('Amp4')

In [ ]:
thisPulse_thirdCrystal = copy.deepcopy(thisPulse_telescope2)
num_passes = 3

for prop_index in np.arange(num_passes):
    print('Pass ', prop_index+1)
    
    thisPulse_thirdCrystal = e_drift_srw.propagate(thisPulse_thirdCrystal)
    
    # Crystal
    thisPulse_thirdCrystal = crystal_3.propagate(thisPulse_thirdCrystal, prop_type, gain, radial_n2)
    plot_excited_states(crystal_3)
    
    thisPulse_thirdCrystal = e_drift_srw.propagate(thisPulse_thirdCrystal)
    
    if prop_index < (num_passes -1):
        thisPulse_thirdCrystal.ideal_mirror_180()
    
    central_wavelength[8 + prop_index] = extract_central_wavelength(thisPulse_thirdCrystal)

plot_all(thisPulse_thirdCrystal)
print(thisPulse_thirdCrystal.calc_total_energy())

***
## Final Visualization

In [ ]:
experimental_central_wavelength = np.array([800.3,800.5,800.9,803,805.2,805.6,806.6,808,809.5,811.2,812.4])

plt.figure()
plt.plot(pass_number, central_wavelength, 'k', label='Simulation')
plt.plot(pass_number, experimental_central_wavelength, 'r', label='Experiment')
plt.xlabel('Pass Number')
plt.ylabel('Central Wavelength (nm)')
plt.legend()
plt.tick_params(direction="in")
plt.grid()
plt.show()

In [ ]:
amp = np.array([0,1,2,3])

wavelength_0 = (const.h * const.c)/(const.e * photon_e_ev_init) * 1e9
wavelength_1 = (const.h * const.c)/(const.e * photon_e_ev_first) * 1e9
wavelength_2 = (const.h * const.c)/(const.e * photon_e_ev_second) * 1e9
wavelength_3 = (const.h * const.c)/(const.e * photon_e_ev_final) * 1e9

frequency_0 = (2.0*np.pi*photon_e_ev_init*const.e)/const.h
frequency_1 = (2.0*np.pi*photon_e_ev_first*const.e)/const.h
frequency_2 = (2.0*np.pi*photon_e_ev_second*const.e)/const.h
frequency_3 = (2.0*np.pi*photon_e_ev_final*const.e)/const.h

wavelength = np.array([np.sum(wavelength_0*weight_0),np.sum(wavelength_1*weight_1),np.sum(wavelength_2*weight_2),np.sum(wavelength_3*weight_3)])
frequency = np.array([np.sum(frequency_0*weight_0),np.sum(frequency_1*weight_1),np.sum(frequency_2*weight_2),np.sum(frequency_3*weight_3)])

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Pulse Location in 100TW Amplifier')
ax1.set_ylabel('Mean Wavelength (nm)', color=color)
ax1.plot(amp, wavelength, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xticks([0,1,2,3])
ax1.set_xticklabels(['Initial', '1st Amp', '2nd Amp', '3rd Amp'])

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Mean Angular Frequency (rad/s)', color=color) 
ax2.plot(amp, frequency, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
photon_e_ev_init = np.zeros(thisPulse_initial.nslice)
photon_e_ev_first = np.zeros(thisPulse_firstCrystal.nslice)
photon_e_ev_beamsplitter = np.zeros(thisPulse_beamsplitter.nslice)
photon_e_ev_second = np.zeros(thisPulse_secondCrystal.nslice)
photon_e_ev_final= np.zeros(thisPulse_thirdCrystal.nslice)

photon_number_init = np.zeros(thisPulse_initial.nslice)
photon_number_first = np.zeros(thisPulse_firstCrystal.nslice)
photon_number_beamsplitter = np.zeros(thisPulse_beamsplitter.nslice)
photon_number_second = np.zeros(thisPulse_secondCrystal.nslice)
photon_number_final= np.zeros(thisPulse_thirdCrystal.nslice)

for laser_index_i in np.arange(thisPulse_initial.nslice):
    photon_number_init[laser_index_i] = np.sum(thisPulse_initial.slice[laser_index_i].n_photons_2d.mesh)
    photon_number_first[laser_index_i] = np.sum(thisPulse_firstCrystal.slice[laser_index_i].n_photons_2d.mesh)
    photon_number_beamsplitter[laser_index_i] = np.sum(thisPulse_beamsplitter.slice[laser_index_i].n_photons_2d.mesh)
    photon_number_second[laser_index_i] = np.sum(thisPulse_secondCrystal.slice[laser_index_i].n_photons_2d.mesh)
    photon_number_final[laser_index_i]= np.sum(thisPulse_thirdCrystal.slice[laser_index_i].n_photons_2d.mesh)
    
    photon_e_ev_init[laser_index_i] = thisPulse_initial.slice[laser_index_i].photon_e_ev
    photon_e_ev_first[laser_index_i] = thisPulse_firstCrystal.slice[laser_index_i].photon_e_ev
    photon_e_ev_beamsplitter[laser_index_i] = thisPulse_beamsplitter.slice[laser_index_i].photon_e_ev
    photon_e_ev_second[laser_index_i] = thisPulse_secondCrystal.slice[laser_index_i].photon_e_ev
    photon_e_ev_final[laser_index_i]= thisPulse_thirdCrystal.slice[laser_index_i].photon_e_ev

weight_0 = photon_number_init / np.sum(photon_number_init)
weight_1 = photon_number_first / np.sum(photon_number_first)
weight_2 = photon_number_second / np.sum(photon_number_second)
weight_3 = photon_number_final / np.sum(photon_number_final)

wavelength_0ex = np.append(850,np.append(wavelength_0,760))
wavelength_1ex = np.append(850,np.append(wavelength_1,760))
wavelength_2ex = np.append(850,np.append(wavelength_2,760))
wavelength_3ex = np.append(850,np.append(wavelength_3,760))

weight_0ex = np.append(0,np.append(weight_0,0))
weight_1ex = np.append(0,np.append(weight_1,0))
weight_2ex = np.append(0,np.append(weight_2,0))
weight_3ex = np.append(0,np.append(weight_3,0))

plt.figure(figsize=(4.6*2.0,3.6*1.2))
plt.plot(wavelength_0ex, weight_0ex/np.max(weight_0ex), 'g', label='Regen Out')
plt.plot(wavelength_1ex, weight_1ex/np.max(weight_1ex) + 0.2, 'k', label='Amp2 Out')
plt.plot(wavelength_2ex, weight_2ex/np.max(weight_2ex) + 0.4, 'r', label='Amp3 Out')
plt.plot(wavelength_3ex, weight_3ex/np.max(weight_3ex) + 0.6, 'b', label='Amp4 Out')
plt.xlabel(r'Wavelength (nm)')
plt.ylabel(r'Spectrum Data')
plt.legend()
plt.title('Study of Red-Shifting Versus Amplification')
plt.tick_params(direction="in")
plt.xlim(760,850)
plt.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
slice_index = np.arange(num_laser_slices)

color = 'tab:blue'
ax1.set_xlabel('Slice Index')
ax1.set_ylabel('Wavelength (nm)', color=color)
ax1.plot(slice_index, (const.h * const.c)/(photon_e_ev_init*const.e)*(1e9), color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Angular Frequency (rad/s)', color=color) 
ax2.plot(slice_index, (2.0*np.pi*photon_e_ev_init*const.e)/const.h, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
pulse_energy_0 = photon_number_init * photon_e_ev_init * const.e
pulse_energy_1 = photon_number_first * photon_e_ev_first * const.e
beamsplitter_energy = photon_number_beamsplitter * photon_e_ev_beamsplitter * const.e
pulse_energy_2 = photon_number_second * photon_e_ev_second * const.e
pulse_energy_3 = photon_number_final * photon_e_ev_final * const.e

amp = np.array([0,1,2,3,4])
plt.figure()
ax = plt.gca()
plt.semilogy(amp, 1.0e3*np.array([np.sum(pulse_energy_0),np.sum(pulse_energy_1),np.sum(beamsplitter_energy),np.sum(pulse_energy_2),np.sum(pulse_energy_3)]), 'k')
plt.xlabel(r'Pulse Location in 100TW Amplifier')
plt.ylabel(r'Pulse Energy (mJ)')
# plt.legend()
plt.title('Simulated Amplification')
plt.tick_params(direction="in")
ax.set_xticks([0,1,2,3,4])
ax.set_xticklabels(['Initial', '1st Amp', 'Beamsplitter', '2nd Amp', '3rd Amp'])
plt.grid()
plt.show()

In [ ]:
1.0e3*np.array([np.sum(pulse_energy_0),np.sum(pulse_energy_1),np.sum(beamsplitter_energy),np.sum(pulse_energy_2),np.sum(pulse_energy_3)])